**Author**: Lulof Pirée

**Date**: June 2022

--------------------------------------------------------------------------------
Copyright (C) 2022 Lulof Pirée

This file is part of the snakes_spn program.

This program is free software:
you can redistribute it and/or modify it under the terms
of the GNU General Public License as published by the Free Software Foundation,
either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY;
without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE.
See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.
If not, see <https://www.gnu.org/licenses/>.

--------------------------------------------------------------------------------
## File content
Main experiment: influence of the following variables
on the amount of inflammation-repression:
* $x_1$: Initial concentration (amount of tokens) of GbPdn.
* $x_2$: Rate constant of GbPdn cleaving ($GbPdn+Gba2 \rightarrow Pdn + Gba2$).
* $x_3$: Rate constant of Gr* unbinding ($Gr^* \rightarrow Gr + Pdn$).
* $x_4$: Rate constant of GrGbPdn unbinding ($GrGbPdn \rightarrow Gr + GbPdn$).

In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from scipy import stats
import math
from numbers import Number

from spn_case_study.launch_experiment import (
    setup_init_markings_all_choices, setup_rates_all_choices)
from spn_case_study.run_grid_search import run_experiment
from spn_case_study.petrinet import PLACES, VARS, TRANS, TRANS_TO_PLACES
from spn_tools.run_simulation import (plot_results, load_log,
    aggregate_dataset_in_timeboxes)
import numpy as np

## Constants

In [2]:
INIT_TOKEN_COUNT = 1000
# Amount of independent repetitions per parameter setup
NUM_REPETITIONS = 50

# Maximum of discrete transition-firings per simulation run
MAX_NUM_TRANSITIONS = 10000
# Maximum amount of simulated time passed before simulation is terminated
MAX_TIME_PASSED = 0.005

## Directory structure

All files should be saved into `main_exp_data`.
This contains 3 subdirectories:
* `const_unbind_rates`: for experiments that fix $x_3$ and $x_4$,
    but try many different values for $x_1$ and $x_2$.
    Contains, for each value $x_3 = CCCC$ 
    and each value of $x_4 = DDDD$
    a subdirectory `x3_CCCC__x4_DDDD`.
    Each such subdirecory contains a directory
    for each variation of $x_1 = AAAA$ and $x_2 = BBBB$,
    e.g., `x1_123e-4__x2_321e-2`.
* `vary_unbind_rates`: same as `const_unbind_rates`,
    but with $(x_3, x_4)$ swapped with $(x_1, x_2)$.
* `baseline`: for the single experiment that has $x_1=x_2=x_3=x_4=0$.
    Used an unrepressed inflammation baseline 
    to assess strength of inflammation-repression.

```
main_exp_data/

    const_unbind_rates/
        x3_CCCC__x4_DDDD/
            x1_AAAA__x2_BBBB/
                logs.json
                hyperparameters.json
            .../
        .../

    vary_unbind_rates/
        x1_AAAA__x2_BBBB/
            x3_CCCC__x4_DDDD/
                logs.json
                hyperparameters.json
            .../
        .../

    baseline/
        logs.json
        hyperparameters.json
```

In [3]:
MAIN_DIR_NAME = "main_exp_dir"
CONST_UNBIND_DIR_NAME = "const_unbind_rates"
VARYING_UNBIND_DIR_NAME = "vary_unbind_rates"
BASELINE_DIR_NAME = "baseline"

# Notebooks usually set the cwd to the directory containing the notebook itself.
print(f"Current directory: {os.getcwd()}")
if "main_experiment.ipynb" in os.listdir(os.getcwd()):
    MAIN_SAVE_DIR = os.path.join(os.getcwd(), MAIN_DIR_NAME)
else:
    raise RuntimeError("Wrong working directory.")

CONST_UNBIND_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, CONST_UNBIND_DIR_NAME)
VARYING_UNBIND_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, VARYING_UNBIND_DIR_NAME)
BASELINE_SAVE_DIR = os.path.join(MAIN_DIR_NAME, BASELINE_DIR_NAME)

all_subdirs = [CONST_UNBIND_SAVE_DIR, VARYING_UNBIND_SAVE_DIR,
                BASELINE_SAVE_DIR]
for dir_name in all_subdirs:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

Current directory: /home/nifrec/Vault/Documents/master_1/bio_modelling/model_ass/snakes_spn/spn_case_study


## Baseline experiment

In [4]:
def check_experiment_already_done(dirname: str) -> bool:
    """
    Return whether is already a data-file of experimental data
    (a `logs.json` file) in the given directory.
    """
    dir_contents = os.listdir(dirname)
    if "logs.json" in dir_contents:
        print("Data for experiment '{dirname}' already exists.")
        return True
    return False

if not check_experiment_already_done(MAIN_SAVE_DIR):
    init_markings = {name: (0,) for name in PLACES}
    init_markings["neutrophil_free"] = (INIT_TOKEN_COUNT,)
    init_markings["infl_sig"] = (INIT_TOKEN_COUNT,)

    rates = {name : "0" for name in TRANS}
    rates["recruit_neutrophil"] = f"{VARS['infl_sig']}*{VARS['neutrophil_free']}"

    run_experiment(rates, init_markings, MAIN_SAVE_DIR, NUM_REPETITIONS, 
                        max_steps=MAX_NUM_TRANSITIONS, max_time=MAX_TIME_PASSED)

Starting repeated experiment with 50 repetitions.
Finished repetition 1/50
Finished repetition 2/50
Finished repetition 3/50
Finished repetition 4/50
Finished repetition 5/50
Finished repetition 6/50
Finished repetition 7/50
Finished repetition 8/50
Finished repetition 9/50
Finished repetition 10/50
Finished repetition 11/50
Finished repetition 12/50
Finished repetition 13/50
Finished repetition 14/50
Finished repetition 15/50
Finished repetition 16/50
Finished repetition 17/50
Finished repetition 18/50
Finished repetition 19/50
Finished repetition 20/50
Finished repetition 21/50
Finished repetition 22/50
Finished repetition 23/50
Finished repetition 24/50
Finished repetition 25/50
Finished repetition 26/50
Finished repetition 27/50
Finished repetition 28/50
Finished repetition 29/50
Finished repetition 30/50
Finished repetition 31/50
Finished repetition 32/50
Finished repetition 33/50
Finished repetition 34/50
Finished repetition 35/50
Finished repetition 36/50
Finished repetition 37/

## First experiment: $x_3, x_4$ fixed

Values to try:
* $x_1 \in [10, 100, 1000, 2000, 3000]$
* $x_2 \in [0.001, 0.01, 1, 10, 100]$